In [16]:
import requests
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, datetime, timedelta, timezone
from scipy.cluster.hierarchy import linkage, dendrogram
from time import sleep
from random import randint

# # API Camp to Camp - 

In [2]:
# #Export data from C2C API

url = 'https://api.camptocamp.org/outings'
france = '&a=14274'
req_activities = {'?act=skitouring':310,'?act=rock_climbing':330,'?act=mountain_climbing':100,'?act=snow_ice_mixed':100,'?act=ice_climbing':100,'?act=hiking':100}

request_num = 0
result=[]

for k,v in req_activities.items():
    for i in range(0,v):
        if i == 0 : 
            api_path = url + k + france
        else:
            api_path = url + k + france + '&offset=' + str(i*30)            
        print(api_path)
        r = requests.get(api_path)
        result.append(r.json())
        request_num += 1
        sleep(randint(1,3))
        
print("Nombre total de reqûetes :" + str(request_num))
        
path = 'C:/Users/Mossuz Damien/Documents/Python Scripts/01 - C2C API/input_/'

with open(path + "merged_file.json", "w",encoding='utf-8') as outfile:
     json.dump(result, outfile, ensure_ascii=False,indent=4)

#Import merged results -        
with open(path + "merged_file.json", "r",encoding='utf-8') as infile:
     json_data = json.load(infile)

#Check - 
print(len(json_data))
print(json_data)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Mossuz Damien/Documents/Python Scripts/01 - C2C API/input_/merged_file.json'